## Stacking을 위한 패키지 vecstack

@ http://j.mp/34Etidr & http://j.mp/2NJQ9NO

### <span style='color:red;'>아래 명령어들로 xgboost & vecstack를 설치하고 Jupyter notebook 재부팅(Kernel Restart)을 진행해주세요</span>
- 설치 중 에러가 발생하면 cmd(명령프롬프트) 우클릭 & 관리자권한으로 실행 후 명령어를 입력해 설치합니다.

In [1]:
!pip install xgboost==1.5.2

  Using cached xgboost-1.5.2-py3-none-win_amd64.whl (106.6 MB)


In [2]:
!pip install vecstack==0.4.0

  Using cached vecstack-0.4.0.tar.gz (18 kB)
  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19880 sha256=83a03ce153657fa1a703c0f567536ff006d90b681ac0ca338204bdeba87a9755
  Stored in directory: c:\users\tech2_07\appdata\local\pip\cache\wheels\17\89\0b\21d5484cbf713c95b641ec1bdc40dd7ae798cbdea2337e3535
Successfully built vecstack


<br>
<br>

## 1. Usage. Functional API

In [3]:
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import accuracy_score 

from sklearn.ensemble import ExtraTreesClassifier 
from sklearn.ensemble import RandomForestClassifier 
from xgboost import XGBClassifier 

import warnings                 
warnings.filterwarnings('ignore')

In [4]:
from vecstack import stacking

In [5]:
# Load demo data 

iris = load_iris() 
X, y = iris.data, iris.target 

In [10]:
# Make train/test split 
# 8:2
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, random_state = 0)

In [11]:
# Initialize 1-st level models. 
# Caution! All models and parameter values are just 
# demonstrational and shouldn't be considered as recommended. 

models = [ 
    ExtraTreesClassifier(random_state = 0, n_jobs = -1, n_estimators = 100, max_depth = 3), 
    RandomForestClassifier(random_state = 0, n_jobs = -1, n_estimators = 100, max_depth = 3), 
    XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3)] 

In [13]:
# Compute stacking features

# Training data x & y 에 대해 3가지 모델들을 모두 학습시킨 후,
# 1) 해당 모델들이 X_train 에 대해 예측한 결과를 Data point 각각에 대해 묶어서 돌려준다. (S_train)
# : Shape of X_train == (120, 4) -> Shape of S_train == (120, 3) <- 모델 각각의 예측치가 하나의 열
# 2) 해당 모델들이 X_test 에 대해 예측한 결과를 Data point 각각에 대해 묶어서 돌려준다. (S_test)
# : Shape of X_test == (30, 4) -> Shape of S_test == (30, 3) <- 모델 각각의 예측치가 하나의 열

S_train, S_test = stacking(models, 
                           X_train, y_train, X_test, 
                           regression = False, 
                           metric = accuracy_score, 
                           n_folds = 4, stratified = True, shuffle = True, 
                           random_state = 0, verbose = 2) 

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [ExtraTreesClassifier]
    fold  0:  [1.00000000]
    fold  1:  [0.90000000]
    fold  2:  [1.00000000]
    fold  3:  [0.90000000]
    ----
    MEAN:     [0.95000000] + [0.05000000]
    FULL:     [0.95000000]

model  1:     [RandomForestClassifier]
    fold  0:  [0.96666667]
    fold  1:  [0.90000000]
    fold  2:  [1.00000000]
    fold  3:  [0.90000000]
    ----
    MEAN:     [0.94166667] + [0.04330127]
    FULL:     [0.94166667]

model  2:     [XGBClassifier]
[10:32:44] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
    fold  0:  [0.93333333]
[10:32:44] WARNING: C:/Users/Administrator/workspace/x

In [16]:
S_train.shape

(120, 3)

In [18]:
S_test.shape

(30, 3)

In [23]:
# Initialize 2-nd level model 

model = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3, eval_metric='mlogloss') 

In [24]:
# Fit 2-nd level model 
# 3개의 모델이 예측한 결과인 S_train을 Feature(3개의 열)로 하여 y_train을 맞추도록 모델을 학습시킨다.

model = model.fit(S_train, y_train) 

In [27]:
# Predict 
# 앞서 3개의 모델이 예측한 결과인 S_test를 Feature로 하여 y_test를 예측한다.

y_pred = model.predict(S_test)

In [28]:
y_pred.shape

(30,)

In [29]:
# Final prediction score
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score: [0.96666667]


<br>
<br>

## 2. Usage. Scikit-learn API (권장)

In [30]:
from vecstack import StackingTransformer

In [31]:
# Get your data

iris = load_iris() 
X, y = iris.data, iris.target 

X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size = 0.2, random_state = 0)

In [32]:
# Initialize 1st level estimators

estimators = [ 
    ('ExtraTrees', ExtraTreesClassifier(random_state = 0, n_jobs = -1, n_estimators = 100, max_depth = 3)),
    ('RandomForest', RandomForestClassifier(random_state = 0, n_jobs = -1, n_estimators = 100, max_depth = 3)),
    ('XGB', XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3, eval_metric='mlogloss'))]

In [33]:
# Initialize StackingTransformer

stack = StackingTransformer(estimators, 
                            regression = False, 
                            metric = accuracy_score, 
                            n_folds = 4, stratified = True, shuffle = True, 
                            random_state = 0, verbose = 2) 

In [36]:
# from sklearn.preprocessing import StandardScaler
# sc = StandardScaler()
# sc.fit(X_train) # X.train의 각 열에 대한 mean & std 계산
# x_train_scaled = sc.transform(X_train)
# y_train_scaled = sc.transform(Y_train)

# Fit
stack = stack.fit(X_train, y_train) # train x & y를 넣으면서 estimators에 있는 1st stage 모델들을 학습

task:         [classification]
n_classes:    [3]
metric:       [accuracy_score]
variant:      [A]
n_estimators: [3]

estimator  0: [ExtraTrees: ExtraTreesClassifier]
    fold  0:  [1.00000000]
    fold  1:  [0.90000000]
    fold  2:  [1.00000000]
    fold  3:  [0.90000000]
    ----
    MEAN:     [0.95000000] + [0.05000000]

estimator  1: [RandomForest: RandomForestClassifier]
    fold  0:  [0.96666667]
    fold  1:  [0.90000000]
    fold  2:  [1.00000000]
    fold  3:  [0.90000000]
    ----
    MEAN:     [0.94166667] + [0.04330127]

estimator  2: [XGB: XGBClassifier]
    fold  0:  [0.93333333]
    fold  1:  [0.90000000]
    fold  2:  [1.00000000]
    fold  3:  [0.90000000]
    ----
    MEAN:     [0.93333333] + [0.04082483]



In [38]:
# Get your stacked features

S_train = stack.transform(X_train) # == predict
S_test = stack.transform(X_test) # == predict

Train set was detected.
Transforming...

estimator  0: [ExtraTrees: ExtraTreesClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [RandomForest: RandomForestClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  2: [XGB: XGBClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

Transforming...

estimator  0: [ExtraTrees: ExtraTreesClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  1: [RandomForest: RandomForestClassifier]
    model from fold  0: done
    model from fold  1: done
    model from fold  2: done
    model from fold  3: done
    ----
    DONE

estimator  2: [XGB: XGBClass

In [39]:
# Use 2nd level estimator with stacked features

model = XGBClassifier(seed = 0, n_jobs = -1, learning_rate = 0.1, n_estimators = 100, max_depth = 3, eval_metric='mlogloss') 
model = model.fit(S_train, y_train)

y_pred = model.predict(S_test) 
print('Final prediction score: [%.8f]' % accuracy_score(y_test, y_pred))

Final prediction score: [0.96666667]
